# Feature 생성

## 음식물 쓰레기 데이터
- 날짜-시-동으로 묶어서 합한 후 파일로 저장

In [1]:
import pandas as pd

df = pd.read_csv('01_제공데이터_update(210811)/01_음식물쓰레기_FOOD_WASTE_210811_update.CSV', encoding='cp949')

FileNotFoundError: ignored

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df2 = df.groupby(['base_date', 'city', 'emd_nm']).sum()

In [ ]:
df2.head()

### index를 column으로

In [ ]:
df2.reset_index(inplace=True)

In [ ]:
df2

In [ ]:
# df2.to_csv('음식물쓰레기.csv', encoding='cp949', index=None)

## 내국인 유동 인구
- visit vs not visit
- 여성 vs 남성
- ageband

In [ ]:
pop_in = pd.read_csv('01_제공데이터_update(210811)/02-1_내국인유동인구_KOREAN-002.CSV', encoding='cp949')
pop_in.head()

In [ ]:
pop_in.info()

In [ ]:
ageband = pop_in['age'].unique()
sex = pop_in['sex'].unique()

In [ ]:
from itertools import product
items = [ageband, sex]
all_kind = list(product(*items))

In [ ]:
all_kind

In [ ]:
pop_in2 = pop_in.groupby(['base_date', 'city', 'emd_nm', 'sex', 'age']).sum()

In [ ]:
pop_in2.head(20)

In [ ]:
pop_in2.reset_index(inplace=True)  

In [ ]:
pop_in2.head(20)

In [ ]:
for age, sex in all_kind:
    pop_in2[str(age)+sex+'notvisit'] = pop_in2[['age', 'sex', 'resd_pop_cnt', 'work_pop_cnt']].apply(lambda x: x[2]+x[3] if (x[0]==age and x[1]==sex) else 0, axis=1)
    pop_in2[str(age)+sex+'visit'] = pop_in2[['age', 'sex', 'visit_pop_cnt']].apply(lambda x: x[2] if (x[0]==age and x[1]==sex) else 0, axis=1)

In [ ]:
pop_in2.head(20)

In [ ]:
pop_in2 = pop_in2.groupby(['base_date', 'city', 'emd_nm']).sum()

In [ ]:
pop_in2.head(20)

In [ ]:
pop_in2.reset_index(inplace=True)  

In [ ]:
pop_in2.drop(columns=['age','time', 'emd_cd'], inplace=True)

In [ ]:
pop_in2

In [ ]:
pop_in2.drop(columns=['resd_pop_cnt', 'work_pop_cnt', 'visit_pop_cnt'], inplace=True)

In [ ]:
# pop_in2.to_csv('내국인_vnv.csv', encoding='cp949', index=None)

In [ ]:
len(df2['emd_nm'].unique())

In [ ]:
len(pop_in2['emd_nm'].unique())

## 외국인 유동인구

In [ ]:
pop_f_l = pd.read_csv('01_제공데이터_update(210811)/02-2_장기체류 외국인 유동인구_LONG_TERM_FRGN.CSV', encoding='cp949')

In [ ]:
pop_f_l.info()

In [ ]:
pop_f_l.head()

In [ ]:
pop_f_l2 = pop_f_l.groupby(['base_date', 'city', 'emd_nm']).sum()

In [ ]:
pop_f_l2.reset_index(inplace=True)

In [ ]:
pop_f_l2['장기체류외국인'] = pop_f_l2.iloc[:, -3:].sum(axis=1)

In [ ]:
pop_f_l2

In [ ]:
pop_f_l2 = pop_f_l2[['base_date', 'city', 'emd_nm', '장기체류외국인']]

In [ ]:
pop_f_l2

In [ ]:
# pop_f_l2.to_csv('장기체류외국인.csv', encoding='cp949', index=None)

In [ ]:
pop_f_s = pd.read_csv('01_제공데이터_update(210811)/02-3_단기체류 외국인 유동인구_SHORT_TERM_FRGN.CSV', encoding='cp949')

In [ ]:
pop_f_s.head()

In [ ]:
pop_f_s = pop_f_s.groupby(['base_date', 'city', 'emd_nm']).sum()

In [ ]:
pop_f_s.reset_index(inplace=True)

In [ ]:
pop_f_s.head()

In [ ]:
pop_f_s = pop_f_s[['base_date', 'city', 'emd_nm', 'visit_pop_cnt']]

In [ ]:
pop_f_s.rename(columns={'visit_pop_cnt':'단기체류외국인'}, inplace=True)

In [ ]:
pop_f_s

In [ ]:
# pop_f_s.to_csv('단기체류외국인.csv', encoding='cp949', index=None)

In [ ]:
pop_f = pd.merge(pop_f_l2, pop_f_s, how='left', on=['base_date', 'city', 'emd_nm'])

In [ ]:
pop_f

In [ ]:
pop_f.isnull().sum()

In [ ]:
pop_f = pop_f.fillna(0)

In [ ]:
pop_f

In [ ]:
# pop_f.to_csv('장단기체류외국인.csv', encoding='cp949', index=None)

## 데이터 합치기

In [ ]:
df_all = pd.merge(pop_in2, pop_f, how='outer', on=['base_date', 'city', 'emd_nm'])

In [ ]:
df_all = pd.merge(df_all, df2, how='outer', on=['base_date', 'city', 'emd_nm'])

In [ ]:
df_all

In [ ]:
df_all.drop(columns=['pay_amt'], inplace=True)
df_all.rename(columns={'em_cnt':'배출건수', 'em_g':'배출량'}, inplace=True)

In [ ]:
df_all.to_csv('음식물+유동인구.csv', encoding='cp949', index=None)

In [ ]:
df_all

### Null 처리
- 특정 날짜의 지역이 알수없음인 유동인구를, 해당 날짜의 배출건수와 배출량이 Null인 지역의 합으로 대체

In [ ]:
df_all[df_all['배출건수'].isnull()]

In [ ]:
df_all[df_all['배출건수'].isnull()].groupby('base_date').sum().reset_index()

In [ ]:
df_fill = df_all[df_all['배출건수'].isnull()].groupby('base_date').sum().reset_index()

In [ ]:
df_fill

In [ ]:
df_all[df_all['city']=='알수없음']

In [ ]:
df_all[df_all['city']=='알수없음'].dropna(axis=1)

In [ ]:
df_unknown = df_all[df_all['city']=='알수없음'].dropna(axis=1)

In [ ]:
pd.merge(df_unknown, df_fill.iloc[:,:-2], how='outer', on='base_date')

In [ ]:
df_all[~df_all.isnull().any(axis=1)]

In [ ]:
pd.concat([pd.merge(df_unknown, df_fill.iloc[:,:-2], how='outer', on='base_date'), df_all[~df_all.isnull().any(axis=1)]])

In [ ]:
df_allall = pd.concat([pd.merge(df_unknown, df_fill.iloc[:,:-2], how='outer', on='base_date'), df_all[~df_all.isnull().any(axis=1)]])

In [ ]:
df_allall

In [ ]:
df_allall.isnull().sum()

In [ ]:
df_allall.sort_values(by=['base_date']).to_csv('음식물+유동인구.csv', encoding='cp949', index=None)

In [ ]:
df_a = df_allall.dropna(axis=0)

In [ ]:
df_a.sort_values(by=['base_date']).to_csv('음식물+유동인구+null처리.csv', encoding='cp949', index=None)